# Iterated CPR Games

This is a template that will allow you to start a cadCAD experiment without needing to fork another project.

## Check cadCAD

This cell doesn't do anything, but it does ensure that you know what version of cadCAD you're running. That way if/when you encounter technical difficulties you can tell the community which version of cadCAD you're running. Might save you hours of pain if all you need to do is upgrade to the latest version. 

In [11]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /usr/local/lib/python3.8/site-packages
Requires: funcy, fn, pathos, pandas
Required-by: cadCAD-diagram
Note: you may need to restart the kernel to use updated packages.


## Import Stuff

These are the libraries you'll need (cadCAD stuff) and that could be useful (python stuff) for your cadCAD model.

We're using [cadCAD Diagram](https://github.com/cadCAD-org/cadCAD_diagram), which is separate from cadCAD. If you're using the VS Code container from this repo it's already installed, but otherwise you might need to instal it.

In [12]:
# Python
import numpy as np
import random as random

# cadCAD
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor
from cadCAD_diagram import diagram_from_config

# Notebook setup
experiment = Experiment()
del configs[:]

## Initial Params

These are the parameters that control the behavior of a system. For example, if you were modeling a Web3 protocol these would be the parameters within a system that token holders could vote to modify. All of the parameters of a protocol that can be modified should be accessible here. That way it's easy to modify and run similations to explore how parameter changes might affect the network.

In [13]:
# The initial state of the system
genesis_states = {
    # A state variable
    'agents': 10,
    # Another state variable
    'cpr': 100
}

# The parameters to run the model
params = {
    # Params within the system
    'M': {
        'agent_appropriation': [1],
        'cpr_respawn': [10]
    },
    # Timesteps per run
    'T': range(100),
    # Number of parallel runs
    'N': 1
}

## Policy Functions

Policy functions are like flows in stock and flow diagrams. They modify the inputs to state update functions.

In [14]:
# Policy to appropriate the CPR
def p_appropriate(params, step, sH, s):
    # Variables
    aa = params['agent_appropriation']
    num_agents = s['agents']

    # Logic
    total_appropriation = aa * num_agents

    # Output
    return ({'total_appropriation': total_appropriation})

# Respawn the CPR
def p_respawn(params, step, sH, s):
    # Variables
    cpr_respawn = params['cpr_respawn']

    # Logic
    # CPR is boosted at the respawn rate
    cpr_boost = cpr_respawn

    # Output
    return ({'cpr_boost': cpr_boost})

## State Update Functions

These functions take in inputs (state variables and policies) and modify the state.

In [15]:
# Appropriate the CPR based on the appropriation policy
def s_appropriate(params, step, sH, s, _input):
    # Variables
    y = 'cpr'
    x = s[y]
    
    # Logic
    # CPR diminishes at the rate of immediate agent appropriation
    appropriate_cpr = _input['total_appropriation']
    x -= appropriate_cpr
    
    # Output
    return (y, x)

# Respawn the CPR based on the respawn policy
def s_respawn(params, step, sH, s, _input):
    # Variables
    y = 'cpr'
    x = s[y]

    # Logic
    # CPR repawns every timestep
    cpr_boost = _input['cpr_boost']
    x += cpr_boost

    # Output
    return (y, x)

## State Update Block

If you're approaching cadCAD from Web3 you can think of the state of the model as something that evolves in blocks. Each block has a set of actions that updates the state. Those transactions then get batched into blocks to be processed together. In cadCAD blocks are called "`partial_state_update_blocks`." As you can see below, this is an array that is very similar to a "block" in a blockchain in that it represents a set of actions to update the state. That state is then updated across many timesteps. This is similar to how the state of a blockchain is updated over many timesteps as new blocks are added.



In [16]:
partial_state_update_blocks = [
    {
        'label': 'Appropriate',
        # policies represent the behavior of agents that interact with components of the system
        'policies': { 
            'total_appropriation': p_appropriate
        },
        # state variables that will be updated
        'variables': {
            'cpr': s_appropriate
        }
    },
    {
        'label': 'Respawn',
        'policies': {
            'cpr_boost': p_respawn
        },
        'variables': {
            'cpr': s_respawn
        }
    }
]

## Running the cadCAD Engine

In [17]:
experiment.append_configs(
    initial_state = genesis_states,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = config_sim(params)
)

exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs) 
raw_system_events, tensor_field, sessions = simulation.execute() 


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 2, 1, 2)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.05s


## Data Visualization

This is often half the battle. Not only do need to design and build a cadCAD model, but you need to understand how it's working and be able to effectively communicate that to other people. A picture says a thousand words, thus enter data viz. Getting good at using python data viz libraries is probably the highest leverage thing you can do after you learn the cadCAD basics.

In [18]:
diagram_from_config(configs[0])

In [19]:
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

agents  cpr  simulation
subset run timestep substep                         
0      1   0        0            10  100           0
           1        1            10   90           0
                    2            10  100           0
           2        1            10   90           0
                    2            10  100           0
...                             ...  ...         ...
           98       2            10  100           0
           99       1            10   90           0
                    2            10  100           0
           100      1            10   90           0
                    2            10  100           0

[201 rows x 3 columns]

In [20]:
simulation_result.plot('timestep', ['agents', 'cpr'], grid=True, 
        colormap = 'gist_rainbow',
        xticks=list(simulation_result['timestep'].drop_duplicates()), 
        yticks=list(range(1+(simulation_result['agents']+simulation_result['cpr']).max())))

<AxesSubplot:xlabel='timestep'>